# Inventory Management Code Demo

One potential application of reinforcement learning involves ordering supplies with mutliple suppliers having various lead times and costs in order to meet a changing demand. Lead time in inventory management is the lapse in time between when an order is placed to replenish inventory and when the order is received. This affects the amount of stock a supplier needs to hold at any point in time. Moreover, due to having multiple suppliers, at every stage the supplier is faced with a decision on how much to order from each supplier, noting that more costly suppliers might have to be used to replenish the inventory from a shorter lead time.

The inventory control model addresses this by modeling an environment where there are multiplie suppliers with different costs and lead times. Orders must be placed with these suppliers to have an on-hand inventory to meet a changing demand. However, both having supplies on backorder and holding unused inventory have associated costs. The goal of the agent is to choose the amount to order from each supplier to maximize the revenue earned.

### Package Installation
First we import the necessary packages

In [2]:
import or_suite
import numpy as np

import copy

import os
from stable_baselines3.common.monitor import Monitor
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
import pandas as pd


import gym

## Run Simulation with One Supplier

### Environment Parameters

The inventory management problem has several parameters. Here we use some simple values for an environment with only 1 supplier:
* `lead_times`: array of ints representing the lead times of each supplier
* `demand_dist`: the random number sampled from the given distribution to be used to calculate the demand
* `supplier_costs`: array of ints representing the costs of each supplier
* `hold_cost`: The int holding cost.
* `backorder_cost`: The backorder holding cost.
* `max_inventory`: The maximum value (int) that can be held in inventory
* `max_order`: The maximum value (int) that can be ordered from each supplier
* `epLen`: The int number of time steps to run the experiment for.
* `starting_state`: An int list containing enough indices for the sum of all the lead times, plus an additional index for the initial on-hand inventory.
* `neg_inventory`: A bool that says whether the on-hand inventory can be negative or not.
* `nEps`: an int representing the number of episodes
* `numIters`: an int representing the number of iterations

In [3]:
CONFIG = {'lead_times': [5],
           'demand_dist': lambda x: np.random.poisson(5),
           'supplier_costs': [10],
           'hold_cost': 1,
           'backorder_cost': 100,
           'max_inventory': 1000,
           'max_order': 50,
           'epLen': 500,
           'starting_state': None,
           'neg_inventory': False
         }
CONFIG['epLen'] = 100
epLen = CONFIG['epLen']
nEps = 2
numIters = 10


### Experimental parameters

Next we need to specify parameters for the inventory control experiment.
* `seed` allows random numbers to be generated. 
* `dirPath`, a string, is the location where the data files are stored.
* `deBug`, a bool, prints information to the command line when set true. 
* `save_trajectory`, a bool, saves the trajectory information of the simulation when set to true. 
* `render` renders the algorithm when set to true.
* `pickle` is a bool that saves the information to a pickle file when set to true.

In [4]:
DEFAULT_SETTINGS = {'seed': 1, 
                    'recFreq': 1, 
                    'dirPath': '../data/ambulance/', 
                    'deBug': False, 
                    'nEps': nEps, 
                    'numIters': numIters, 
                    'saveTrajectory': True, 
                    'epLen' : CONFIG['epLen'],
                    'render': False,
                    'pickle': False
                    }

env = gym.make('MultipleSuppliers-v0', config=CONFIG)
mon_env = Monitor(env)

### Specifying Agent
We specify 6 different agents to compare the effectiveness of each.
* `SB PPO` is Proximal Policy Optimization. When policy is updated, there is a parameter that “clips” each policy update so that action update does not go too far
* `Random` implements the randomized RL algorithm, which chooses random amounts to order from each supplier between 0 and the $maxorder$ value.
* `TBS` is an agent that uses an order-up-to-amount, $S$, and for the supplier with the largest lead time, orders $S$ minus the current inventory. For the other suppliers, different values are used, which are stored in an array, $r$, which has the length of the number of suppliers minus 1.

In [5]:
agents = { # 'SB PPO': PPO(MlpPolicy, mon_env, gamma=1, verbose=0, n_steps=epLen),
'Random': or_suite.agents.rl.random.randomAgent(),
'TBS': or_suite.agents.inventory_control_multiple_suppliers.base_surge.base_surgeAgent([],0)
}

### Running Algorithm

Run the different heuristics in the environment

In [6]:
path_list_line = []
algo_list_line = []
path_list_radar = []
algo_list_radar= []

#each index of param_list is another list, param, where param[0] is r and param[1] is S
max_order = CONFIG['max_order']
param_list = []
for S in range(max_order + 1):
        param_list.append([[],S])
        
for agent in agents:
    print(agent)
    DEFAULT_SETTINGS['dirPath'] = '../data/inventory_control_'+str(agent)+'/'
    if agent == 'SB PPO':
        or_suite.utils.run_single_sb_algo(mon_env, agents[agent], DEFAULT_SETTINGS)
    elif agent == 'TBS':
        or_suite.utils.run_single_algo_tune(env, agents[agent], param_list, DEFAULT_SETTINGS)
    else:
        or_suite.utils.run_single_algo(env, agents[agent], DEFAULT_SETTINGS)

    path_list_line.append('../data/inventory_control_'+str(agent))
    algo_list_line.append(str(agent))
    if agent != 'SB PPO':
        path_list_radar.append('../data/inventory_control_'+str(agent))
        algo_list_radar.append(str(agent))

Random
Writing to file data.csv
TBS
Chosen parameters: [[], 26]
Writing to file data.csv
[[], 26]


### Generate Figures

Create a chart to compare the different heuristic functions

In [7]:
fig_path = '../figures/'
fig_name = 'inventory'+'_line_plot'+'.pdf'
or_suite.plots.plot_line_plots(path_list_line, algo_list_line, fig_path, fig_name, int(nEps / 40)+1)

additional_metric = {}
fig_name = 'inventory'+'_radar_plot'+'.pdf'
or_suite.plots.plot_radar_plots(path_list_radar, algo_list_radar,
fig_path, fig_name,
additional_metric
)

  Algorithm   Reward      Time    Space
0    Random -98526.2  4.396521 -93336.3
1       TBS -14378.4  4.545949 -93252.8


In [8]:
from IPython.display import IFrame
IFrame("../figures/inventory_line_plot.pdf", width=600, height=280)

In [9]:
IFrame("../figures/inventory_radar_plot.pdf", width=600, height=450)

### Results
Here we see that the `TBS` agent performs better than the `Random` agent for an environment that involves only a single supplier.

## Run Simulation with 2 Suppliers


### Experimental Parameters

The package has default specifications for all of the environments in the file `or_suite/envs/env_configs.py`, and so we use a default for the inventory control problem for 2 suppliers.

In [10]:
CONFIG =  or_suite.envs.env_configs.inventory_control_multiple_suppliers_modified_config
CONFIG['epLen'] = 500
CONFIG['neg_inventory']= False
epLen = CONFIG['epLen']
nEps = 2
numIters = 10
print(epLen)


500


In [11]:
DEFAULT_SETTINGS = {'seed': 1, 
                    'recFreq': 1, 
                    'dirPath': '../data/ambulance/', 
                    'deBug': False, 
                    'nEps': nEps, 
                    'numIters': numIters, 
                    'saveTrajectory': True, 
                    'epLen' : CONFIG['epLen'],
                    'render': False,
                    'pickle': False
                    }

env = gym.make('MultipleSuppliers-v0', config=CONFIG)
mon_env = Monitor(env)

### Specifying Agent

In [12]:
agents = { # 'SB PPO': PPO(MlpPolicy, mon_env, gamma=1, verbose=0, n_steps=epLen),
'Random': or_suite.agents.rl.random.randomAgent(),
'TBS': or_suite.agents.inventory_control_multiple_suppliers.base_surge.base_surgeAgent([14],0)
}

### Running Algorithm

In [13]:
path_list_line = []
algo_list_line = []
path_list_radar = []
algo_list_radar= []

#each index of param_list is another list, param, where param[0] is r and param[1] is S
max_order = CONFIG['max_order']
param_list = []
for r in range(max_order+1):
    for S in range(max_order + 1):
        param_list.append([[r],S])
        
for agent in agents:
    print(agent)
    DEFAULT_SETTINGS['dirPath'] = '../data/inventory_control_'+str(agent)+'/'
    if agent == 'SB PPO':
        or_suite.utils.run_single_sb_algo(mon_env, agents[agent], DEFAULT_SETTINGS)
    elif agent == 'TBS':
        or_suite.utils.run_single_algo_tune(env, agents[agent], param_list, DEFAULT_SETTINGS)
    else:
        or_suite.utils.run_single_algo(env, agents[agent], DEFAULT_SETTINGS)

    path_list_line.append('../data/inventory_control_'+str(agent))
    algo_list_line.append(str(agent))
    if agent != 'SB PPO':
        path_list_radar.append('../data/inventory_control_'+str(agent))
        algo_list_radar.append(str(agent))

Random
Writing to file data.csv
TBS
Chosen parameters: [[9], 12]
Writing to file data.csv
[[9], 12]


### Generate Figures

In [15]:
fig_path = '../figures/'
fig_name = 'inventory'+'_line_plot'+'.pdf'
or_suite.plots.plot_line_plots(path_list_line, algo_list_line, fig_path, fig_name, int(nEps / 40)+1)

additional_metric = {}
fig_name = 'inventory'+'_radar_plot'+'.pdf'
or_suite.plots.plot_radar_plots(path_list_radar, algo_list_radar,
fig_path, fig_name,
additional_metric
)

  Algorithm    Reward      Time     Space
0    Random -550071.1  2.192443 -505358.0
1       TBS  -91386.2  2.474930 -505353.6


In [17]:
IFrame("../figures/inventory_line_plot.pdf", width=600, height=280)

In [18]:
IFrame("../figures/inventory_radar_plot.pdf", width=600, height=450)

### Results
Here we see that the `TBS` agent also performs better than the `Random` agent for an environment that involves two suppliers.

## Run with 3 Suppliers

We now use an example environment that uses 3 suppliers, each with different lead times and costs. This results in a non-trivial best action when using the `TBS` agent that still performs better than the `Random` agent.

### Experimental Parameters

In [19]:
CONFIG = {'lead_times': [5, 7, 11],
           'demand_dist': lambda x: np.random.poisson(17),
           'supplier_costs': [100 ,85, 73],
           'hold_cost': 1,
           'backorder_cost': 200,
           'max_inventory': 1000,
           'max_order': 20,
           'epLen': 500,
           'starting_state': None,
           'neg_inventory': False
         }
CONFIG['epLen'] = 100
epLen = CONFIG['epLen']
nEps = 2
numIters = 10

In [ ]:
DEFAULT_SETTINGS = {'seed': 1, 
                    'recFreq': 1, 
                    'dirPath': '../data/ambulance/', 
                    'deBug': False, 
                    'nEps': nEps, 
                    'numIters': numIters, 
                    'saveTrajectory': True, 
                    'epLen' : CONFIG['epLen'],
                    'render': False,
                    'pickle': False
                    }

env = gym.make('MultipleSuppliers-v0', config=CONFIG)
mon_env = Monitor(env)

### Specifying Agents

In [20]:
agents = { # 'SB PPO': PPO(MlpPolicy, mon_env, gamma=1, verbose=0, n_steps=epLen),
'Random': or_suite.agents.rl.random.randomAgent(),
'TBS': or_suite.agents.inventory_control_multiple_suppliers.base_surge.base_surgeAgent([14,14],0)
}

### Running Algorithm

In [21]:
path_list_line = []
algo_list_line = []
path_list_radar = []
algo_list_radar= []

#each index of param_list is another list, param, where param[0] is r and param[1] is S
max_order = CONFIG['max_order']
param_list = []
for S in range(max_order + 1):
    for r1 in range(max_order + 1):
        for r2 in range(max_order +1):
            param_list.append([[r1, r2],S])
        
for agent in agents:
    print(agent)
    DEFAULT_SETTINGS['dirPath'] = '../data/inventory_control_'+str(agent)+'/'
    if agent == 'SB PPO':
        or_suite.utils.run_single_sb_algo(mon_env, agents[agent], DEFAULT_SETTINGS)
    elif agent == 'TBS':
        or_suite.utils.run_single_algo_tune(env, agents[agent], param_list, DEFAULT_SETTINGS)
    else:
        or_suite.utils.run_single_algo(env, agents[agent], DEFAULT_SETTINGS)

    path_list_line.append('../data/inventory_control_'+str(agent))
    algo_list_line.append(str(agent))
    if agent != 'SB PPO':
        path_list_radar.append('../data/inventory_control_'+str(agent))
        algo_list_radar.append(str(agent))

Random
Writing to file data.csv
TBS


AssertionError: Action, [0 0 0],  not part of action space

### Generate Figures

In [ ]:
fig_path = '../figures/'
fig_name = 'inventory'+'_line_plot'+'.pdf'
or_suite.plots.plot_line_plots(path_list_line, algo_list_line, fig_path, fig_name, int(nEps / 40)+1)

additional_metric = {}
fig_name = 'inventory'+'_radar_plot'+'.pdf'
or_suite.plots.plot_radar_plots(path_list_radar, algo_list_radar,
fig_path, fig_name,
additional_metric
)

In [ ]:
IFrame("../figures/inventory_line_plot.pdf", width=600, height=280)

In [ ]:
IFrame("../figures/inventory_radar_plot.pdf", width=600, height=450)

### Results
Here we see that the `TBS` agent also performs better than the `Random` agent for an environment that involves three suppliers.

## Nontrivial Two Suppliers Example

We now use an example environment that uses 2 suppliers, each with different lead times and costs, but where the resulting optimal action for the TBS agent does not have `r=0` or `S=0`.

### Experimental Parameters

In [ ]:
CONFIG = {'lead_times': [5, 7],
           'demand_dist': lambda x: np.random.poisson(17),
           'supplier_costs': [100 ,85],
           'hold_cost': 1,
           'backorder_cost': 200,
           'max_inventory': 1000,
           'max_order': 20,
           'epLen': 500,
           'starting_state': None,
           'neg_inventory': False
         }
CONFIG['epLen'] = 100
epLen = CONFIG['epLen']
nEps = 2
numIters = 10

In [ ]:
DEFAULT_SETTINGS = {'seed': 1, 
                    'recFreq': 1, 
                    'dirPath': '../data/ambulance/', 
                    'deBug': False, 
                    'nEps': nEps, 
                    'numIters': numIters, 
                    'saveTrajectory': True, 
                    'epLen' : CONFIG['epLen'],
                    'render': False,
                    'pickle': False
                    }

env = gym.make('MultipleSuppliers-v0', config=CONFIG)
mon_env = Monitor(env)

### Specifying Agents

In [ ]:
agents = { # 'SB PPO': PPO(MlpPolicy, mon_env, gamma=1, verbose=0, n_steps=epLen),
'Random': or_suite.agents.rl.random.randomAgent(),
'TBS': or_suite.agents.inventory_control_multiple_suppliers.base_surge.base_surgeAgent([14,14],0)
}

### Running Algorithms

In [ ]:
path_list_line = []
algo_list_line = []
path_list_radar = []
algo_list_radar= []

#each index of param_list is another list, param, where param[0] is r and param[1] is S
max_order = CONFIG['max_order']
param_list = []
for r in range(max_order+1):
    for S in range(max_order + 1):
        param_list.append([[r],S])
        
for agent in agents:
    print(agent)
    DEFAULT_SETTINGS['dirPath'] = '../data/inventory_control_'+str(agent)+'/'
    if agent == 'SB PPO':
        or_suite.utils.run_single_sb_algo(mon_env, agents[agent], DEFAULT_SETTINGS)
    elif agent == 'TBS':
        or_suite.utils.run_single_algo_tune(env, agents[agent], param_list, DEFAULT_SETTINGS)
    else:
        or_suite.utils.run_single_algo(env, agents[agent], DEFAULT_SETTINGS)

    path_list_line.append('../data/inventory_control_'+str(agent))
    algo_list_line.append(str(agent))
    if agent != 'SB PPO':
        path_list_radar.append('../data/inventory_control_'+str(agent))
        algo_list_radar.append(str(agent))

### Generate Figures

In [ ]:
fig_path = '../figures/'
fig_name = 'inventory'+'_line_plot'+'.pdf'
or_suite.plots.plot_line_plots(path_list_line, algo_list_line, fig_path, fig_name, int(nEps / 40)+1)

additional_metric = {}
fig_name = 'inventory'+'_radar_plot'+'.pdf'
or_suite.plots.plot_radar_plots(path_list_radar, algo_list_radar,
fig_path, fig_name,
additional_metric
)

In [ ]:
IFrame("../figures/inventory_line_plot.pdf", width=600, height=280)

In [ ]:
IFrame("../figures/inventory_radar_plot.pdf", width=600, height=450)

### Results
Here, the `TBS` agent still performs better than the `Random` agent. However, the `r` and `S` values for the `TBS` agent are both non-zero.